## Задание 1.

Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: - - .

In [1]:
import json
import requests
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from time import sleep
import urllib3
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def habrdate2datetime(habrdate:str):
    """
    Преобразование дат статей Habr в формат datetime
    
    Входные параметры:
        habrdate -- дата статьи Хабра в формате строки 
                    Примеры: 'сегодня в 04:05', '2 октября 2020 в 18:02'
                    
    Возвращаемое значение: datetime. В случае ошибки возвращаеся 1 Января 1900
    """
    
    habrdate = habrdate.lower()
    
    # Возвращаемое значение функции, инициализируем его датой 1 Января 1900
    # Отдаем такую дату в случае ошибок функции
    dt_habrdate = datetime(1900, 1, 1)
    
    # Пробуем найти часы и минуты
    re_post_time = re.compile(r'\d\d:\d\d$').findall(habrdate)
    if len(re_post_time) == 0:
        return dt_habrdate

    # Сохраняем часы и минуты в формате HH:MM
    post_time = re_post_time[0]

    # Если в дате есть слово 'сегодня' или 'вчера', то сразу формируем datetime 
    # за сегодняшнюю или вчерашнюю дату
    if any([word in habrdate for word in ('сегодня', 'вчера')]):
        time = datetime.now() if 'сегодня' in habrdate else datetime.now() - timedelta(1)
        dt_habrdate = datetime(time.year,time.month, 
                               time.day, int(post_time.split(':')[0]), int(post_time.split(':')[1]))
        
    #В случае если в дате указано русское название месяца, то меняем на английское значение
    # и конвертируем дату в datetime
    else:
        
        if 'января' in habrdate:
            habrdate = habrdate.replace('января', 'January')
        elif 'февраля' in habrdate:
            habrdate = habrdate.replace('февраля', 'February')
        elif 'марта' in habrdate:
            habrdate = habrdate.replace('марта', 'March')
        elif 'апреля' in habrdate:
            habrdate = habrdate.replace('апреля', 'April')
        elif 'мая' in habrdate:
            habrdate = habrdate.replace('мая', 'May')
        elif 'июня' in habrdate:
            habrdate = habrdate.replace('июня', 'June')
        elif 'июля' in habrdate:
            habrdate = habrdate.replace('июля', 'Jule')
        elif 'августа' in habrdate:
            habrdate = habrdate.replace('августа', 'August')
        elif 'сентября' in habrdate:
            habrdate = habrdate.replace('сентября', 'September')
        elif 'октября' in habrdate:
            habrdate = habrdate.replace('октября', 'October')
        elif 'ноября' in habrdate:
            habrdate = habrdate.replace('ноября', 'November')
        elif 'декабря' in habrdate:
            habrdate = habrdate.replace('декабря', 'December')

        # С помощью регулярного выражения находим дату в формате ДЕНЬ НАЗВАНИЕ МЕСЯЦА ГОД
        re_date = re.compile(r'\d{1,2}\s\w+\s\d{4}').findall(habrdate)
        if len(re_date) == 0:
            return dt_habrdate
        
        # Формируем datetime из полученных данных
        try:
            dt_habrdate = datetime.strptime(re_date[0]+ f" {post_time.split(':')[0]}:{post_time.split(':')[1]}", 
                                            f"%d %B %Y %H:%M") 
        except:
            return dt_habrdate 
        
    return dt_habrdate

In [3]:
def get_habr_posts_by_keywords(keywords:list, pages:int=1):
    """
    Функция для поиска постов с определенными ключевыми словами в тексте.
    
    Входные параметры:
      keywords -- список с ключевыми словами, например, ['python', 'парсинг']
      pages -- количество страниц для анализа
      
    Возвращаемое значение: датафрейм Pandas
    """
    
    df = pd.DataFrame()
    
    for page_number in range(1, pages+1):
        
        # получаем страницу с указанным номером с самыми свежими постами
        web_response = requests.get(f'https://habr.com/ru/all/page{page_number}/', verify=False)
        bs = BeautifulSoup(web_response.text, 'html.parser')
        
        # Все блоки постов находятся в тегах article с классом post
        posts = bs.find_all('article', class_='post')
        
        # Перебираем все найденные посты на странице
        for post in posts:
            
            # Получаем объект bs с заголовком поста
            post_title = post.find('a', class_='post__title_link')
            
            # Получаем объект bs с кратким описанием поста
            post_description = post.find('div', class_='post__text')
            
            # Создаем общий текст для анализа заголовок + описание
            analyse_text = (post_title.text + post_description.text).lower()
            
            # Если объекты заголовка и краткого описания поста не найдены, считаем это некорректным вариантом
            if not post_title or not post_description:
                # Чтобы поймать ошибки разного форматирования сделаем принт для диагностики проблемы
                print(f"Не удалось найти заголовок|краткое описание статьи habr")
                continue
            
            # Если хоть одно ключевое слово содержится в заголовке и описании, то добавляем пост в датафрейм
            if any([keyword.lower() in analyse_text for keyword in keywords]):

                # Получаем ссылку на полную версию поста
                post_link = post_title.get('href')

                # Получаем дату размещения
                date_post = post.find('span', class_='post__time')

                # Если найдена дата поста, то пробуем ее преобразовать в формат datetime
                if date_post:
                    date_post = habrdate2datetime(date_post.text)
        
                row = {'page': page_number, 
                       'date': date_post,
                       'title': post_title.text.strip(), 
                       'link': post_link}    
                df = pd.concat([df, pd.DataFrame([row])])

        # Задержка перед парсингом следующей страницы
        sleep(0.5)
        
    # Добавляем корректный индекс в итоговый датафрейм
    df.index = [i for i in range(len(df))]
    
    return df

In [4]:
KEYWORDS = ['python', 'парсинг']
df_posts = get_habr_posts_by_keywords(KEYWORDS, pages=5)
display(df_posts)

,page,date,title,link
0,1,2020-10-05 23:31:00,Выход стабильной версии Python 3.9.0,https://habr.com/ru/post/522170/
1,1,2020-10-05 18:26:00,Разработка графического профайлера Python Func...,https://habr.com/ru/company/skillfactory/blog/...
2,1,2020-10-05 17:27:00,Serverless телеграм бот с использованием Яндек...,https://habr.com/ru/post/522126/
3,2,2020-10-05 16:11:00,Отношение один к одному: связывание модели пол...,https://habr.com/ru/company/otus/blog/522106/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: - - - <текст статьи>

In [5]:
def get_habr_posts_by_keywords(keywords:list, pages:int=5):
    """
    Функция для поиска постов с определенными ключевыми словами в тексте.
    
    Входные параметры:
      keywords -- список с ключевыми словами, например, ['python', 'парсинг']
      pages -- количество страниц для анализа
      
    Возвращаемое значение: датафрейм Pandas
    """

    df = pd.DataFrame()
    
    for page_number in range(1, pages+1):
        
        # получаем страницу с указанным номером с самыми свежими постами
        web_response = requests.get(f'https://habr.com/ru/all/page{page_number}/', verify=False)
        bs = BeautifulSoup(web_response.text, 'html.parser')
        
        # Все блоки постов находятся в тегах article с классом post
        posts = bs.find_all('article', class_='post')
        
        # Перебираем все найденные посты на странице
        for post in posts:
            
            # Флаг post_is_matched сигнализирует о том, что в данных поста найденны ключевые слова.
            # Здесь происходит инициализация переменной, во время анализа поста может измениться на True
            post_is_matched = False
            
            # Получаем объект bs с заголовком поста
            post_title = post.find('a', class_='post__title_link')
            
            # Получаем объект bs с кратким описанием поста
            post_description = post.find('div', class_='post__text')
            
            # Получаем ссылку на полную версию поста
            post_link = post_title.get('href')
            
            # Получаем дату размещения
            date_post = post.find('span', class_='post__time')
            
            # Если объекты заголовка, краткого описания поста или ссылка на пост не найдены, 
            # считаем это некорректным вариантом и пропускаем
            if not post_title or not post_description or not post_link:
                # Чтобы поймать ошибки разного форматирования сделаем принт для диагностики проблемы
                print(f"Не удалось найти заголовок|краткое описание|ссылку статьи habr")
                continue
                                     
            # Создаем общий текст для анализа заголовок + описание
            analyse_text = (post_title.text + post_description.text).lower()
            
            # Если хоть одно ключевое слово содержится в заголовке и описании, то добавляем пост в датафрейм
            if any([keyword.lower() in analyse_text for keyword in keywords]):
                post_is_matched = True 
             
            # Если в краткой версии поста не найдены ключевые слова, пробуем открыть полную версию поста
            # и искать ключевые слова в полной версии
            else:
                # Задержка перед парсингом следующей страницы
                sleep(0.5)
                
                web_response = requests.get(post_link, verify=False)
                bs = BeautifulSoup(web_response.text, 'html.parser')
                
                # Полная версия поста находится в контейнере div с классом post__body
                post_body_full = bs.find('div', class_='post__body')  
                
                if not post_body_full:
                    # Чтобы поймать ошибки разного форматирования сделаем принт для диагностики проблемы
                    print(f"Не найден полный текст поста по ссылке {post_link}")
                    continue
                    
                # Если хоть одно ключевое слово содержится в тексте полного поста, 
                # то добавляем его в датафрейм    
                if any([keyword.lower() in post_body_full.text for keyword in keywords]):
                    post_is_matched = True
                 
            # Если нашли клчюевые слова в полном текте статьи или сокращенном, то добавляем данные
            # в датафрейм
            if post_is_matched:
                
                # Если найдена дата поста, то пробуем ее преобразовать в формат datetime
                if date_post:
                    date_post = habrdate2datetime(date_post.text)
                
                row = {'page': page_number, 
                       'date': date_post,
                       'title': post_title.text.strip(), 
                       'link': post_link}    
                df = pd.concat([df, pd.DataFrame([row])])
                
        # Задержка перед парсингом следующей страницы      
        sleep(0.5)
      
    # Добавляем корректный индекс в итоговый датафрейм
    df.index = [i for i in range(len(df))]
    
    return df

In [6]:
KEYWORDS = ['python', 'парсинг']
df_posts = get_habr_posts_by_keywords(KEYWORDS, pages=5)
display(df_posts)

,page,date,title,link
0,1,2020-10-05 23:31:00,Выход стабильной версии Python 3.9.0,https://habr.com/ru/post/522170/
1,1,2020-10-05 18:26:00,Разработка графического профайлера Python Func...,https://habr.com/ru/company/skillfactory/blog/...
2,1,2020-10-05 17:27:00,Serverless телеграм бот с использованием Яндек...,https://habr.com/ru/post/522126/
3,2,2020-10-05 16:11:00,Отношение один к одному: связывание модели пол...,https://habr.com/ru/company/otus/blog/522106/
4,3,2020-10-05 12:26:00,Http-stubs — поиск идеального инструмента,https://habr.com/ru/company/umatech/blog/521722/
5,5,2020-10-04 12:48:00,Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/digital-ecosystems...
6,5,2020-10-03 22:41:00,Токсичность Белорусского IT аутсорса,https://habr.com/ru/post/521928/


## Задание 2

Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [7]:
def str2date(date):
    """
    Функция для конвертации даты в формате строки в формат datetime. Функция возвращает
    дату в формате datetime. Если в переменной str_date указан некорректный формат даты, 
    то функция выдаст исключение.
    date - дата в формате YYYY-MM-DD
    """
    
    known_formats = [
    '%A, %B %d, %Y', #  The Moscow Times - Wednesday, October 2, 2002
    '%A, %d.%m.%y',  #  The Guardian - Friday, 11.10.13
    '%A, %d %B %Y',  #  Daily News - Thursday, 18 August 1977
    '%Y-%m-%d',      #  Формат YYYY-MM-DD
    ]
    
    for datetime_format in known_formats:
        try:
            return datetime.strptime(date, datetime_format).date()
        except:
            pass
    raise ValueError('date содержит неподдерживаемый формат времени')

In [8]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}
data = {'emailAddresses': EMAIL}
response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)

if response.status_code != 200 or not response.text:
    message = f"Во время полученя данных с API произошла ошибка"
    print(message)   
else:
    df_breaches = pd.DataFrame()
    response_json = json.loads(response.text)
    for email, breaches_dict in response_json['summary'].items():
        for breach_id in breaches_dict['breaches']:   
            row = {'email': email,
                   'date':  str2date(response_json['breaches'][str(breach_id)]['publishDate'][:10]),
                   'source': response_json['breaches'][str(breach_id)]['site'],
                   'description': response_json['breaches'][str(breach_id)]['description']
                  }
            df_breaches = pd.concat([df_breaches, pd.DataFrame([row])])
    
            
    # Добавляем корректный индекс в итоговый датафрейм
    df_breaches.index = [i for i in range(len(df_breaches))]
 
    display(df_breaches)

,email,date,source,description
0,xxx@x.ru,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2020-09-10,na,This is a compilation of files including breac...
6,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
7,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
8,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
9,yyy@y.com,2019-03-28,verifications.io,Big data e-mail verification platform verifica...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'

In [9]:
TOKEN = ''
# Количество получаемых постов
COUNT_POSTS = 100
# Домен сообщества, например, для сообщества https://vk.com/onyxboox_russia домен onyxboox_russia
GROUP_DOMAIN  = 'onyxboox_russia'
# Версия API
api_version = '5.124'

url = 'https://api.vk.com/method/wall.get'
params = {
    'domain': GROUP_DOMAIN,
    'count': COUNT_POSTS, 
    'access_token': TOKEN,
    'v': api_version
}

json_api_response = json.loads(requests.post(url, data=params, verify=False).text)

In [10]:
# Первый метод формирования датафрейма, более быстрый
if json_api_response.get('error'):
    message = f"Во время полученя данных с API произошла ошибка: {json_api_response['error']['error_msg']}"
    print(message)   
else:
    # Создадим датафрейм со столбцами Дата, Текст поcта
    dates = []
    texts = []
    for post in json_api_response['response']['items']:
        dates.append(post.get('date'))
        texts.append(post.get('text'))

    df_vk_posts = pd.DataFrame({'date': dates, 'text': texts})
    # Преоборазовываем unixtime в datetime
    df_vk_posts.date = pd.to_datetime(df_vk_posts.date, unit='s')

    display(df_vk_posts[:10])

,date,text
0,2016-11-07 07:36:40,Только для своих! ;) \nСкидка 5% нашим подписч...
1,2020-10-04 13:32:59,"Целая семья букридеров ONYX, включая даже деду..."
2,2020-10-03 17:29:02,В сентябре вышел трейлер «Дюны» Дени Вильнева ...
3,2020-10-02 13:01:57,📖 Брайан Стивенсон «Звонок за ваш счет. Истори...
4,2020-10-01 16:17:51,"Poke 2 – маленький, да удаленький! \nМодель им..."
5,2020-09-30 15:52:59,Американские ученые провели исследование и выя...
6,2020-09-29 12:17:02,"Борис Акунин рассказал, что подготовил для дет..."
7,2020-09-28 16:44:10,"То чувство, когда человеческий портрет не пере..."
8,2020-09-27 14:41:56,"Я любитель привидений. Я никогда не слыхал, чт..."
9,2020-09-26 14:48:16,📖 Мадлен Миллер «Цирцея». \n\nМадлен Миллер – ...


In [11]:
# Второй метод формирования датафрейма, работает в 2 раа медленнее предыдущего
if json_api_response.get('error'):
    message = f"Во время полученя данных с API произошла ошибка: {json_api_response['error']['error_msg']}"
    print(message)   
else:
    df_vk_posts = pd.DataFrame(json_api_response['response']['items']).loc[:, ['date', 'text']]
    # Преоборазовываем unixtime в datetime
    df_vk_posts.date = pd.to_datetime(df_vk_posts.date, unit='s')

    display(df_vk_posts[:10])

,date,text
0,2016-11-07 07:36:40,Только для своих! ;) \nСкидка 5% нашим подписч...
1,2020-10-04 13:32:59,"Целая семья букридеров ONYX, включая даже деду..."
2,2020-10-03 17:29:02,В сентябре вышел трейлер «Дюны» Дени Вильнева ...
3,2020-10-02 13:01:57,📖 Брайан Стивенсон «Звонок за ваш счет. Истори...
4,2020-10-01 16:17:51,"Poke 2 – маленький, да удаленький! \nМодель им..."
5,2020-09-30 15:52:59,Американские ученые провели исследование и выя...
6,2020-09-29 12:17:02,"Борис Акунин рассказал, что подготовил для дет..."
7,2020-09-28 16:44:10,"То чувство, когда человеческий портрет не пере..."
8,2020-09-27 14:41:56,"Я любитель привидений. Я никогда не слыхал, чт..."
9,2020-09-26 14:48:16,📖 Мадлен Миллер «Цирцея». \n\nМадлен Миллер – ...
